## Data loading

### Subtask:
Load the data from "wc-product-export-20-3-2025-1742505370284.csv" into a pandas DataFrame.


**Reasoning**:
Load the data from the CSV file into a pandas DataFrame.



In [ ]:
import pandas as pd

df = pd.read_csv('')
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Semestre 7/wc-product-export-20-3-2025-1742505370284.csv'

## Data exploration

### Subtask:
Explore the loaded data to understand its structure, identify key features, and understand the distribution of variables.


**Reasoning**:
Explore the basic structure and descriptive statistics of the dataframe to understand the data.



In [ ]:
# Data Shape
print(f"The DataFrame has {df.shape[0]} rows and {df.shape[1]} columns.")

# Data Types
print("\nData Types of Each Column:\n", df.dtypes)

# Descriptive Statistics for Numerical Features
numerical_features = df.select_dtypes(include=['number'])
print("\nDescriptive Statistics for Numerical Features:\n", numerical_features.describe())

# Missing Values
print("\nMissing Values in Each Column:\n", df.isnull().sum())

# Unique Values for Categorical Features
categorical_features = df.select_dtypes(include=['object'])
for column in categorical_features.columns:
  print(f"\nUnique values for {column}: {df[column].nunique()}")

**Reasoning**:
Analyze the distribution of key variables such as 'Precio normal' and 'Precio rebajado' using histograms.



In [ ]:
import matplotlib.pyplot as plt

# Distribution of 'Precio normal'
plt.figure(figsize=(8, 6))
plt.hist(df['Precio normal'], bins=20)
plt.xlabel('Precio normal')
plt.ylabel('Frequency')
plt.title('Distribution of Precio normal')
plt.show()

# Distribution of 'Precio rebajado'
plt.figure(figsize=(8, 6))
plt.hist(df['Precio rebajado'], bins=20)
plt.xlabel('Precio rebajado')
plt.ylabel('Frequency')
plt.title('Distribution of Precio rebajado')
plt.show()

**Reasoning**:
Calculate and visualize the correlation matrix between numerical features to identify potential relationships.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Correlation Matrix
correlation_matrix = numerical_features.corr()

# Visualize the Correlation Matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

**Reasoning**:
Analyze the trends and patterns over time if there is a date/time column in the dataframe.



In [ ]:
# Check if there is a date/time column
datetime_columns = [col for col in df.columns if 'Día' in col]

if datetime_columns:
    # Convert the date/time column to datetime objects
    for column in datetime_columns:
        df[column] = pd.to_datetime(df[column], errors='coerce')

    # Analyze trends over time (e.g., sales over time)
    # Assuming 'Día en que empieza el precio rebajado' is a relevant date column
    if 'Día en que empieza el precio rebajado' in df.columns:
        sales_over_time = df.groupby('Día en que empieza el precio rebajado')['Precio normal'].sum()
        plt.figure(figsize=(12, 6))
        plt.plot(sales_over_time.index, sales_over_time.values)
        plt.xlabel('Date')
        plt.ylabel('Total Sales')
        plt.title('Sales Over Time')
        plt.show()


## Data cleaning

### Subtask:
Clean the data by handling missing values, removing outliers, and dealing with inconsistencies in the data.


**Reasoning**:
Start by handling missing values by calculating the percentage of missing values in each column and then decide on strategies for dealing with them based on the percentage.



In [ ]:
# Calculate the percentage of missing values in each column
missing_percentage = df.isnull().sum() * 100 / len(df)

# Print the percentage of missing values for each column
print("Percentage of missing values in each column:\n", missing_percentage)

# Remove rows with a small percentage of missing values (e.g., less than 5%)
df = df.dropna(thresh=len(df.columns) - int(0.05 * len(df.columns)))

# For columns with a larger percentage of missing values, consider imputation strategies
# Example: Impute missing values in 'Precio normal' with the median
if 'Precio normal' in df.columns and df['Precio normal'].isnull().any():
  df['Precio normal'].fillna(df['Precio normal'].median(), inplace=True)

# Example: Impute missing values in 'Descripción' with the most frequent value
if 'Descripción' in df.columns and df['Descripción'].isnull().any():
  df['Descripción'].fillna(df['Descripción'].mode()[0], inplace=True)


**Reasoning**:
Continue with outlier detection and handling for numerical features.



In [ ]:
# Identify numerical features
numerical_features = df.select_dtypes(include=['number'])

# Loop through numerical features and handle outliers
for column in numerical_features.columns:
  # Check if the column has any data
  if not df[column].empty:
    # Calculate the IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    # Handle outliers (e.g., remove rows containing outliers)
    if not outliers.empty:
      df = df.drop(outliers.index)


**Reasoning**:
Now, we need to deal with inconsistencies in categorical variables. We can start by examining the unique values for each categorical feature and look for spelling errors, inconsistent formatting, or duplicate categories.



In [ ]:
# Identify categorical features
categorical_features = df.select_dtypes(include=['object'])

# Loop through categorical features and handle inconsistencies
for column in categorical_features.columns:
  # Check if the column has any data
  if not df[column].empty:
    # Convert values to lowercase and remove leading/trailing whitespace
    df[column] = df[column].str.lower().str.strip()

    # Replace specific inconsistencies (e.g., misspellings)
    # Example: Replace "bolsas" with "bolsa"
    if 'Nombre' in column:
      df[column] = df[column].replace('bolsas', 'bolsa')

    # Check for duplicate categories and merge them
    unique_values = df[column].unique()
    for value in unique_values:
        # Check if a similar value exists
        similar_values = [val for val in unique_values if val != value and value in val or val in value]
        if similar_values:
            # Replace similar values with the most frequent value
            df[column] = df[column].replace(similar_values, value)


# Data Wrangling



In [ ]:
import pandas as pd

# Load the data again
df = pd.read_csv('/content/drive/MyDrive/Semestre 7/wc-product-export-20-3-2025-1742505370284.csv')

# Impute missing values in 'Precio normal' with the median
if 'Precio normal' in df.columns and df['Precio normal'].isnull().any():
  df['Precio normal'] = df['Precio normal'].fillna(df['Precio normal'].median())

# Impute missing values in 'Precio rebajado' with the median
if 'Precio rebajado' in df.columns and df['Precio rebajado'].isnull().any():
  df['Precio rebajado'] = df['Precio rebajado'].fillna(df['Precio rebajado'].median())

# Impute missing values in 'Inventario' with the median
if 'Inventario' in df.columns and df['Inventario'].isnull().any():
  df['Inventario'] = df['Inventario'].fillna(df['Inventario'].median())


# Feature engineering: Create a new column representing the "product category" based on the product names.
if 'Nombre' in df.columns:
  # Try different methods to extract product categories
  df['product_category'] = df['Nombre'].str.lower().str.extract(r'(camisetas|pantalones|vestidos|zapatos|bolsos)')
  # If the previous method doesn't work, try another one
  df.loc[df['product_category'].isnull(), 'product_category'] = df.loc[df['product_category'].isnull(), 'Nombre'].str.lower().str.split().str[0]
  # If the previous method doesn't work, try another one
  df.loc[df['product_category'].isnull(), 'product_category'] = 'Otros'

# Feature engineering: Create a new feature indicating whether the product is on sale or not.
if 'Precio rebajado' in df.columns and 'Precio normal' in df.columns:
  df['on_sale'] = df['Precio rebajado'] < df['Precio normal']

# Feature engineering: Transform the date columns into features like "month", "day of week", "year" and "quarter" to capture potential seasonality effects.
if 'Día en que empieza el precio rebajado' in df.columns:
  df['Día en que empieza el precio rebajado'] = pd.to_datetime(df['Día en que empieza el precio rebajado'], errors='coerce')
  df['month'] = df['Día en que empieza el precio rebajado'].dt.month
  df['day_of_week'] = df['Día en que empieza el precio rebajado'].dt.dayofweek
  df['year'] = df['Día en que empieza el precio rebajado'].dt.year
  df['quarter'] = df['Día en que empieza el precio rebajado'].dt.quarter

# Data aggregation: Group data by product category and calculate the total sales
if 'product_category' in df.columns and 'Precio normal' in df.columns:
  sales_by_category = df.groupby('product_category')['Precio normal'].sum()
  print(sales_by_category)

# Feature selection: Select the features that are most relevant for the predictive model.
# For simplicity, select a few relevant features based on domain knowledge.
relevant_features = ['Precio normal', 'Precio rebajado', 'Inventario', 'product_category', 'on_sale', 'month', 'day_of_week', 'year', 'quarter']
df_selected = df[relevant_features]

# Display the first few rows of the prepared data
display(df_selected.head())

## Data splitting

### Subtask:
Split the prepared data into training, validation, and testing sets.


**Reasoning**:
Split the `df_selected` dataframe into training, validation, and testing sets using `train_test_split`.



In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
df_train, df_test = train_test_split(df_selected, test_size=0.3, random_state=42)

# Further split the training set into training and validation sets
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

## Model optimization

### Subtask:
Optimize the trained linear regression model to improve its performance.


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Separate the target variable and predictor variables
X_train = df_train.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_train = df_train['Precio normal']
X_val = df_val.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_val = df_val['Precio normal']

# Create a column transformer to one-hot encode categorical features and impute missing values
categorical_features = ['product_category']
numerical_features = ['Precio rebajado', 'Inventario', 'on_sale']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features),
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean'))
        ]), numerical_features)
    ])

# Create a pipeline for the model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Define the hyperparameter grid (remove 'normalize' parameter)
param_grid = {
    'regressor__fit_intercept': [True, False]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the validation set using the best model
y_pred = best_model.predict(X_val)

# Evaluate the model's performance
mae = mean_absolute_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred) ** 0.5

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Best Hyperparameters: {grid_search.best_params_}")

**Reasoning**:
The model performance can be further improved by considering feature engineering. We can create new features that might be more predictive, such as interaction terms or polynomial features.



In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Separate the target variable and predictor variables
X_train = df_train.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_train = df_train['Precio normal']
X_val = df_val.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_val = df_val['Precio normal']

# Create a column transformer to one-hot encode categorical features and impute missing values
categorical_features = ['product_category']
numerical_features = ['Precio rebajado', 'Inventario', 'on_sale']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features),
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean'))
        ]), numerical_features)
    ])

# Create a pipeline for the model with polynomial features
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2)),  # Add polynomial features
    ('regressor', LinearRegression(fit_intercept=False))  # Use the best hyperparameter from previous step
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = pipeline.predict(X_val)

# Evaluate the model's performance
mae = mean_absolute_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred) ** 0.5

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

**Reasoning**:
The model's performance has improved after adding polynomial features. Now, we should analyze the feature importance to understand the impact of different features on the prediction and identify potentially irrelevant features.



In [ ]:
# Get the coefficients of the linear regression model
coefficients = pipeline.named_steps['regressor'].coef_

# Get the feature names after preprocessing
feature_names = pipeline.named_steps['poly'].get_feature_names_out(
    pipeline.named_steps['preprocessor'].get_feature_names_out()
)

# Create a DataFrame to store the coefficients and feature names
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by the absolute value of the coefficients
feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(
    by=['Absolute Coefficient'], ascending=False
)

# Display the feature importance
display(feature_importance)

## Model evaluation

### Subtask:
Evaluate the performance of the optimized linear regression model on the testing dataset.


**Reasoning**:
Evaluate the performance of the optimized linear regression model on the testing dataset by making predictions using the best model found during optimization and calculating the MAE, RMSE, and R2 score.



**Reasoning**:
The `squared=False` argument is not valid for the `mean_squared_error` function in the current version of scikit-learn. To calculate the RMSE, we need to calculate the MSE and then take the square root manually.



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separate the target variable and predictor variables for the test set
X_test = df_test.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_test = df_test['Precio normal']

# Use the best model to make predictions on the testing dataset
y_pred = pipeline.predict(X_test)

# Calculate the evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print the calculated evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

## Data visualization

### Subtask:
Visualize the predicted consumption trends against the actual consumption data.


**Reasoning**:
Create a scatter plot to visualize the model's accuracy in predicting the prices, and a line plot to visualize the actual and predicted prices over time. Since the 'Día en que empieza el precio rebajado' is missing for all the data, we can only plot the scatter plot.



In [ ]:
import matplotlib.pyplot as plt

# Separate the target variable and predictor variables for the test set
X_test = df_test.drop(['Precio normal', 'month', 'day_of_week', 'year', 'quarter'], axis=1)
y_test = df_test['Precio normal']

# Use the best model to make predictions on the testing dataset
y_pred = pipeline.predict(X_test)

# 1. Scatter plot: Actual vs. Predicted 'Precio normal'
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Actual Precio normal")
plt.ylabel("Predicted Precio normal")
plt.title("Actual vs. Predicted Precio normal")
plt.show()

**Reasoning**:
Generate a histogram to visualize the distribution of prediction errors.



In [ ]:
import matplotlib.pyplot as plt

# Calculate prediction errors
errors = y_test - y_pred

# 3. Histogram: Distribution of prediction errors
plt.figure(figsize=(8, 6))
plt.hist(errors, bins=20)
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")
plt.title("Distribution of Prediction Errors")
plt.show()

**Reasoning**:
Create a bar chart to visualize the average sales predicted by the model for different product categories.



In [ ]:
import matplotlib.pyplot as plt

# Assuming 'product_category' and 'Precio normal' are available in df_test
if 'product_category' in df_test.columns and 'Precio normal' in df_test.columns:
    # Group by product category and calculate the mean of 'Precio normal' for actual and predicted values
    mean_actual_sales_by_category = df_test.groupby('product_category')['Precio normal'].mean()
    # Make predictions for the test set
    y_pred = pipeline.predict(X_test)
    df_test['predicted_sales'] = y_pred
    mean_predicted_sales_by_category = df_test.groupby('product_category')['predicted_sales'].mean()

    # Create a bar chart to visualize average sales by category
    plt.figure(figsize=(12, 6))
    width = 0.35
    x = range(len(mean_actual_sales_by_category))
    plt.bar(
        [i - width / 2 for i in x],
        mean_actual_sales_by_category,
        width,
        label='Actual Sales',
    )
    plt.bar(
        [i + width / 2 for i in x],
        mean_predicted_sales_by_category,
        width,
        label='Predicted Sales',
    )
    plt.xticks(x, mean_actual_sales_by_category.index, rotation=45, ha='right')
    plt.xlabel('Product Category')
    plt.ylabel('Average Sales')
    plt.title('Average Predicted Sales vs. Actual Sales by Product Category')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
from datetime import datetime

# Extraemos las categorías de productos conocidas
categorias_entrenadas = df_train['product_category'].unique().tolist()

def predict_sales():
    # Entrada del usuario
    nombre_producto = input("Ingrese el nombre del producto: ").strip().lower()

    # Verificar si el producto pertenece a una categoría conocida
    if nombre_producto not in categorias_entrenadas:
        print(f"Error: El producto '{nombre_producto}' no pertenece a una categoría en la que el modelo fue entrenado.")
        print("Por favor, ingrese un producto de las siguientes categorías:")
        print(", ".join(categorias_entrenadas))
        return  # Terminar la ejecución si la categoría no es válida

    precio_normal = float(input("Ingrese el precio normal del producto: "))
    precio_rebajado = float(input("Ingrese el precio rebajado del producto (si no tiene rebaja, ingrese el mismo precio que el normal): "))
    inventario = int(input("Ingrese el inventario disponible: "))
    fecha_inicio_descuento = input("Ingrese la fecha de inicio del descuento (formato YYYY-MM-DD): ").strip()

    # Determinar si el producto está en rebaja
    en_rebaja = precio_rebajado < precio_normal
    precio_final = precio_rebajado if en_rebaja else precio_normal  # Usar el precio correcto

    # Procesar la fecha de inicio del descuento
    fecha_inicio_descuento = datetime.strptime(fecha_inicio_descuento, "%Y-%m-%d")

    # Crear un DataFrame con los datos proporcionados
    input_data = pd.DataFrame({
        'product_category': [nombre_producto],
        'Precio normal': [precio_normal],
        'Precio rebajado': [precio_rebajado],
        'Precio final': [precio_final],
        'Inventario': [inventario],
        'Día en que empieza el precio rebajado': [fecha_inicio_descuento],
        'on_sale': [en_rebaja]
    })

    # Extraer características de fecha
    input_data['month'] = input_data['Día en que empieza el precio rebajado'].dt.month
    input_data['day_of_week'] = input_data['Día en que empieza el precio rebajado'].dt.dayofweek
    input_data['year'] = input_data['Día en que empieza el precio rebajado'].dt.year
    input_data['quarter'] = input_data['Día en que empieza el precio rebajado'].dt.quarter

    # Hacer la predicción con el modelo entrenado
    predicted_sales = pipeline.predict(input_data)
    ventas_totales = predicted_sales[0] * inventario
    umbral_ventas = 1000  # Ajustar este umbral según los datos históricos

    # Determinar si el producto se venderá bien
    venta_estado = "Este producto no se venderá bien." if ventas_totales > umbral_ventas else "¡Sí, este producto probablemente se venderá!"

    # Mostrar resultados
    print(f"La predicción para el producto '{nombre_producto}' es que las ventas estimadas serán: {predicted_sales[0]:.2f}")
    print(f"Las ventas totales estimadas para este producto son: {ventas_totales:.2f}")
    print(venta_estado)

# Llamar a la función para realizar la predicción
predict_sales()

